# Enter State Farm

In [1]:
from theano.sandbox import cuda
cuda.use('gpu0')

WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: Tesla K80 (CNMeM is enabled with initial size: 95.0% of memory, cuDNN 5103)
WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29



In [2]:
%matplotlib inline
from __future__ import print_function, division
path = "data/state/"
#path = "data/state/sample/"
import utils; reload(utils)
from utils import *
from IPython.display import FileLink

Using Theano backend.


In [3]:
batch_size=64
#batch_size=8

## Setup batches

In [19]:
batches = get_batches(path+'train', batch_size=batch_size)
val_batches = get_batches(path+'valid', batch_size=batch_size*2, shuffle=False)

Found 19893 images belonging to 10 classes.
Found 2531 images belonging to 10 classes.


In [9]:
(val_classes, trn_classes, val_labels, trn_labels, 
    val_filenames, filenames, test_filenames) = get_classes(path)

Found 19893 images belonging to 10 classes.
Found 2531 images belonging to 10 classes.
Found 79726 images belonging to 1 classes.


Rather than using batches, we could just import all the data into an array to save some processing time. (In most examples I'm using the batches, however - just because that's how I happened to start out.)

In [6]:
trn = get_data(path+'train')
val = get_data(path+'valid')

Found 19893 images belonging to 10 classes.
Found 2531 images belonging to 10 classes.


In [8]:
os.mkdir(path + 'results') # if you have'nt created yet

In [7]:
save_array(path+'results/val.dat', val)
save_array(path+'results/trn.dat', trn)

In [6]:
val = load_array(path+'results/val.dat')
trn = load_array(path+'results/trn.dat')

## Re-run sample experiments on full dataset

We should find that everything that worked on the sample (see statefarm-sample.ipynb), works on the full dataset too. Only better! Because now we have more data. So let's see how they go - the models in this section are exact copies of the sample notebook models.

### Single conv layer

In [7]:
def conv1(batches):
    model = Sequential([
            BatchNormalization(axis=1, input_shape=(3,224,224)),
            Convolution2D(32,3,3, activation='relu'),
            BatchNormalization(axis=1),
            MaxPooling2D((3,3)),
            Convolution2D(64,3,3, activation='relu'),
            BatchNormalization(axis=1),
            MaxPooling2D((3,3)),
            Flatten(),
            Dense(200, activation='relu'),
            BatchNormalization(),
            Dense(10, activation='softmax')
        ])

    model.compile(Adam(lr=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit_generator(batches, batches.nb_sample, nb_epoch=2, validation_data=val_batches, 
                     nb_val_samples=val_batches.nb_sample)
    model.optimizer.lr = 0.001
    model.fit_generator(batches, batches.nb_sample, nb_epoch=4, validation_data=val_batches, 
                     nb_val_samples=val_batches.nb_sample)
    return model

In [8]:
model = conv1(batches)
# Accuracy depends on which drivers you picked. Don't be too nervous.

Epoch 1/2
19893/19893 [==============================] - 352s - loss: 0.2277 - acc: 0.9438 - val_loss: 2.1002 - val_acc: 0.3414
Epoch 2/2
19893/19893 [==============================] - 253s - loss: 0.0140 - acc: 0.9987 - val_loss: 1.7023 - val_acc: 0.4405
Epoch 1/4
19893/19893 [==============================] - 259s - loss: 0.0052 - acc: 0.9998 - val_loss: 2.1544 - val_acc: 0.4070
Epoch 2/4
19893/19893 [==============================] - 253s - loss: 0.0049 - acc: 0.9996 - val_loss: 1.7190 - val_acc: 0.4804
Epoch 3/4
19893/19893 [==============================] - 253s - loss: 0.0052 - acc: 0.9994 - val_loss: 1.9867 - val_acc: 0.4409
Epoch 4/4
19893/19893 [==============================] - 253s - loss: 0.0015 - acc: 1.0000 - val_loss: 1.9445 - val_acc: 0.4528


Interestingly, with no regularization or augmentation we're getting some reasonable results from our simple convolutional model. So with augmentation, we hopefully will see some very good results.

### Data augmentation

In [9]:
gen_t = image.ImageDataGenerator(rotation_range=15, height_shift_range=0.05, 
                shear_range=0.1, channel_shift_range=20, width_shift_range=0.1)
batches = get_batches(path+'train', gen_t, batch_size=batch_size)

Found 19893 images belonging to 10 classes.


In [10]:
model = conv1(batches)

Epoch 1/2
19893/19893 [==============================] - 269s - loss: 1.2400 - acc: 0.6039 - val_loss: 1.6553 - val_acc: 0.5065
Epoch 2/2
19893/19893 [==============================] - 261s - loss: 0.6135 - acc: 0.8138 - val_loss: 1.2028 - val_acc: 0.6499
Epoch 1/4
19893/19893 [==============================] - 268s - loss: 0.4204 - acc: 0.8768 - val_loss: 1.3943 - val_acc: 0.6559
Epoch 2/4
19893/19893 [==============================] - 262s - loss: 0.3189 - acc: 0.9122 - val_loss: 1.3435 - val_acc: 0.7033
Epoch 3/4
19893/19893 [==============================] - 262s - loss: 0.2625 - acc: 0.9301 - val_loss: 1.2612 - val_acc: 0.6891
Epoch 4/4
19893/19893 [==============================] - 261s - loss: 0.2192 - acc: 0.9422 - val_loss: 1.2026 - val_acc: 0.7199


In [11]:
model.optimizer.lr = 0.0001
model.fit_generator(batches, batches.nb_sample, nb_epoch=15, validation_data=val_batches, 
                 nb_val_samples=val_batches.nb_sample)

Epoch 1/15
19893/19893 [==============================] - 268s - loss: 0.1951 - acc: 0.9482 - val_loss: 1.3816 - val_acc: 0.6685
Epoch 2/15
19893/19893 [==============================] - 260s - loss: 0.1658 - acc: 0.9588 - val_loss: 1.1443 - val_acc: 0.6812
Epoch 3/15
19893/19893 [==============================] - 261s - loss: 0.1553 - acc: 0.9602 - val_loss: 1.0443 - val_acc: 0.7128
Epoch 4/15
19893/19893 [==============================] - 260s - loss: 0.1301 - acc: 0.9684 - val_loss: 1.0777 - val_acc: 0.7092
Epoch 5/15
19893/19893 [==============================] - 260s - loss: 0.1239 - acc: 0.9688 - val_loss: 1.1360 - val_acc: 0.7199
Epoch 6/15
19893/19893 [==============================] - 261s - loss: 0.1150 - acc: 0.9699 - val_loss: 0.9011 - val_acc: 0.7301
Epoch 7/15
19893/19893 [==============================] - 260s - loss: 0.1058 - acc: 0.9733 - val_loss: 0.8846 - val_acc: 0.7507
Epoch 8/15
19893/19893 [==============================] - 260s - loss: 0.1003 - acc: 0.9737 - val

I'm shocked by *how* good these results are! We're regularly seeing 75-80% accuracy on the validation set, which puts us into the top third or better of the competition. With such a simple model and no dropout or semi-supervised learning, this really speaks to the power of this approach to data augmentation.

### Four conv/pooling pairs + dropout

Unfortunately, the results are still very unstable - the validation accuracy jumps from epoch to epoch. Perhaps a deeper model with some dropout would help.

In [12]:
gen_t = image.ImageDataGenerator(rotation_range=15, height_shift_range=0.05, 
                shear_range=0.1, channel_shift_range=20, width_shift_range=0.1)
batches = get_batches(path+'train', gen_t, batch_size=batch_size)

Found 19893 images belonging to 10 classes.


In [13]:
model = Sequential([
        BatchNormalization(axis=1, input_shape=(3,224,224)),
        Convolution2D(32,3,3, activation='relu'),
        BatchNormalization(axis=1),
        MaxPooling2D(),
        Convolution2D(64,3,3, activation='relu'),
        BatchNormalization(axis=1),
        MaxPooling2D(),
        Convolution2D(128,3,3, activation='relu'),
        BatchNormalization(axis=1),
        MaxPooling2D(),
        Flatten(),
        Dense(200, activation='relu'),
        BatchNormalization(),
        Dropout(0.5),
        Dense(200, activation='relu'),
        BatchNormalization(),
        Dropout(0.5),
        Dense(10, activation='softmax')
    ])

In [14]:
model.compile(Adam(lr=10e-5), loss='categorical_crossentropy', metrics=['accuracy'])

In [15]:
model.fit_generator(batches, batches.nb_sample, nb_epoch=2, validation_data=val_batches, 
                 nb_val_samples=val_batches.nb_sample)

Epoch 1/2
19893/19893 [==============================] - 282s - loss: 2.4874 - acc: 0.2904 - val_loss: 2.1918 - val_acc: 0.2936
Epoch 2/2
19893/19893 [==============================] - 274s - loss: 1.6169 - acc: 0.4883 - val_loss: 1.3755 - val_acc: 0.5946


In [16]:
model.optimizer.lr=0.001

In [17]:
model.fit_generator(batches, batches.nb_sample, nb_epoch=10, validation_data=val_batches, 
                 nb_val_samples=val_batches.nb_sample)

Epoch 1/10
19893/19893 [==============================] - 281s - loss: 1.1847 - acc: 0.6138 - val_loss: 1.1675 - val_acc: 0.7195
Epoch 2/10
19893/19893 [==============================] - 272s - loss: 0.9134 - acc: 0.6983 - val_loss: 1.2071 - val_acc: 0.6906
Epoch 3/10
19893/19893 [==============================] - 273s - loss: 0.7136 - acc: 0.7630 - val_loss: 1.0233 - val_acc: 0.7321
Epoch 4/10
19893/19893 [==============================] - 273s - loss: 0.5905 - acc: 0.8109 - val_loss: 1.1899 - val_acc: 0.7218
Epoch 5/10
19893/19893 [==============================] - 273s - loss: 0.5003 - acc: 0.8352 - val_loss: 1.0730 - val_acc: 0.7298
Epoch 6/10
19893/19893 [==============================] - 274s - loss: 0.4367 - acc: 0.8587 - val_loss: 0.7813 - val_acc: 0.7890
Epoch 7/10
19893/19893 [==============================] - 272s - loss: 0.3748 - acc: 0.8786 - val_loss: 1.0298 - val_acc: 0.7357
Epoch 8/10
19893/19893 [==============================] - 272s - loss: 0.3316 - acc: 0.8951 - val

In [18]:
model.optimizer.lr=0.00001

In [19]:
model.fit_generator(batches, batches.nb_sample, nb_epoch=10, validation_data=val_batches, 
                 nb_val_samples=val_batches.nb_sample)

Epoch 1/10
19893/19893 [==============================] - 280s - loss: 0.2566 - acc: 0.9163 - val_loss: 0.8607 - val_acc: 0.7748
Epoch 2/10
19893/19893 [==============================] - 274s - loss: 0.2377 - acc: 0.9248 - val_loss: 0.9511 - val_acc: 0.7641
Epoch 3/10
19893/19893 [==============================] - 274s - loss: 0.2153 - acc: 0.9326 - val_loss: 0.9779 - val_acc: 0.7349
Epoch 4/10
19893/19893 [==============================] - 271s - loss: 0.2039 - acc: 0.9356 - val_loss: 0.8767 - val_acc: 0.7720
Epoch 5/10
19893/19893 [==============================] - 271s - loss: 0.1943 - acc: 0.9375 - val_loss: 0.8865 - val_acc: 0.7467
Epoch 6/10
19893/19893 [==============================] - 272s - loss: 0.1833 - acc: 0.9427 - val_loss: 0.8146 - val_acc: 0.7653
Epoch 7/10
19893/19893 [==============================] - 271s - loss: 0.1733 - acc: 0.9435 - val_loss: 0.9843 - val_acc: 0.7503
Epoch 8/10
19893/19893 [==============================] - 274s - loss: 0.1591 - acc: 0.9501 - val

This is looking quite a bit better - the accuracy is similar, but the stability is higher. There's still some way to go however...

### Imagenet conv features

Since we have so little data, and it is similar to imagenet images (full color photos), using pre-trained VGG weights is likely to be helpful - in fact it seems likely that we won't need to fine-tune the convolutional layer weights much, if at all. So we can pre-compute the output of the last convolutional layer, as we did in lesson 3 when we experimented with dropout. (However this means that we can't use full data augmentation, since we can't pre-compute something that changes every image.)

In [7]:
vgg = Vgg16()
model=vgg.model
last_conv_idx = [i for i,l in enumerate(model.layers) if type(l) is Convolution2D][-1]
conv_layers = model.layers[:last_conv_idx+1]

In [8]:
conv_model = Sequential(conv_layers) # conv_layers have pretrained weights

In [8]:
# batches shuffle must be set to False when pre-computing features
batches = get_batches(path+'train', batch_size=batch_size, shuffle=False) # shuffle MUST be false

Found 19893 images belonging to 10 classes.


In [9]:
val_batches = get_batches(path+'valid', batch_size=batch_size, shuffle=False) # shuffle MUST be false
test_batches = get_batches(path+'test', batch_size=batch_size, shuffle=False) # shuffle MUST be false

Found 2531 images belonging to 10 classes.
Found 79726 images belonging to 1 classes.


In [10]:
(val_classes, trn_classes, val_labels, trn_labels, 
    val_filenames, filenames, test_filenames) = get_classes(path)

Found 19893 images belonging to 10 classes.
Found 2531 images belonging to 10 classes.
Found 79726 images belonging to 1 classes.


In [ ]:
#conv_feat = conv_model.predict_generator(batches, batches.nb_sample)
#conv_val_feat = conv_model.predict_generator(val_batches, val_batches.nb_sample)
#conv_test_feat = conv_model.predict_generator(test_batches, test_batches.nb_sample)

In [ ]:
#save_array(path+'results/conv_val_feat.dat', conv_val_feat)
#save_array(path+'results/conv_test_feat.dat', conv_test_feat)
#save_array(path+'results/conv_feat.dat', conv_feat)

Let's compute output of the last convolutional layer one by one because it is highly likely that memory error occurs.

In [11]:
conv_feat = conv_model.predict_generator(batches, batches.nb_sample)
save_array(path+'results/conv_feat.dat', conv_feat)

In [12]:
conv_val_feat = conv_model.predict_generator(val_batches, val_batches.nb_sample)
save_array(path+'results/conv_val_feat.dat', conv_val_feat)

In [13]:
conv_test_feat = conv_model.predict_generator(test_batches, test_batches.nb_sample)
save_array(path+'results/conv_test_feat.dat', conv_test_feat)

In [6]:
conv_feat = load_array(path+'results/conv_feat.dat')
conv_val_feat = load_array(path+'results/conv_val_feat.dat')
conv_val_feat.shape

(2531, 512, 14, 14)

### Batchnorm dense layers on pretrained conv layers

Since we've pre-computed the output of the last convolutional layer, we need to create a network that takes that as input, and predicts our 10 classes. Let's try using a simplified version of VGG's dense layers.

In [7]:
def get_bn_layers(p):
    return [
        MaxPooling2D(input_shape=conv_layers[-1].output_shape[1:]), # input_shape=(512, 14, 14)
        Flatten(),
        Dropout(p/2), # why divide by 2 ?
        Dense(128, activation='relu'),
        BatchNormalization(),
        Dropout(p/2),
        Dense(128, activation='relu'),
        BatchNormalization(),
        Dropout(p),
        Dense(10, activation='softmax')
        ]

In [8]:
p=0.8

In [11]:
bn_model = Sequential(get_bn_layers(p))
bn_model.compile(Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [13]:
trn_labels.shape

(19893, 10)

In [14]:
# conv_feat: (19893, 512, 14, 14)
# trn_labels: (19893, 10)

bn_model.fit(conv_feat, trn_labels, batch_size=batch_size, nb_epoch=1, 
             validation_data=(conv_val_feat, val_labels))

Train on 19893 samples, validate on 2531 samples
Epoch 1/1
19893/19893 [==============================] - 5s - loss: 1.4776 - acc: 0.5967 - val_loss: 0.8150 - val_acc: 0.7448


In [15]:
bn_model.optimizer.lr=0.01

In [16]:
bn_model.fit(conv_feat, trn_labels, batch_size=batch_size, nb_epoch=2, 
             validation_data=(conv_val_feat, val_labels))

Train on 19893 samples, validate on 2531 samples
Epoch 1/2
19893/19893 [==============================] - 5s - loss: 0.2483 - acc: 0.9254 - val_loss: 0.8622 - val_acc: 0.7037
Epoch 2/2
19893/19893 [==============================] - 4s - loss: 0.1246 - acc: 0.9663 - val_loss: 0.6622 - val_acc: 0.7949


In [18]:
bn_model.save_weights(path+'models/conv8.h5')

Looking good! Let's try pre-computing 5 epochs worth of augmented data, so we can experiment with combining dropout and augmentation on the pre-trained model.

### Pre-computed data augmentation + dropout

We'll use our usual data augmentation parameters:

In [5]:
gen_t = image.ImageDataGenerator(rotation_range=15, height_shift_range=0.05, 
                shear_range=0.1, channel_shift_range=20, width_shift_range=0.1)
da_batches = get_batches(path+'train', gen_t, batch_size=batch_size, shuffle=False) # shuffle MUST be false

Found 19893 images belonging to 10 classes.


We use those to create a dataset of convolutional features 5x bigger than the training set.

In [9]:
# da_conv_feat = conv_model.predict_generator(da_batches, da_batches.nb_sample*5)
da_conv_feat = conv_model.predict_generator(da_batches, da_batches.nb_sample*1) # use 1 time instead of 5 times to avoid Memory Error...

In [10]:
save_array(path+'results/da_conv_feat2.dat', da_conv_feat)

In [11]:
da_conv_feat = load_array(path+'results/da_conv_feat2.dat')

Let's include the real training data as well in its non-augmented form.

In [12]:
conv_feat = load_array(path+'results/conv_feat.dat')

In [13]:
da_conv_feat = np.concatenate([da_conv_feat, conv_feat])

Since we've now got a dataset 6x bigger than before, we'll need to copy our labels 6 times too.

In [16]:
# da_trn_labels = np.concatenate([trn_labels]*6)
da_trn_labels = np.concatenate([trn_labels]*2)

Based on some experiments the previous model works well, with bigger dense layers.

In [17]:
def get_bn_da_layers(p):
    return [
        MaxPooling2D(input_shape=conv_layers[-1].output_shape[1:]), # input_shape=(512, 14, 14)
        Flatten(),
        Dropout(p),
        Dense(256, activation='relu'),
        BatchNormalization(),
        Dropout(p),
        Dense(256, activation='relu'),
        BatchNormalization(),
        Dropout(p),
        Dense(10, activation='softmax')
        ]

In [4]:
# You don't have to get conv_layers
def get_bn_da_layers(p):
    return [
        MaxPooling2D(input_shape=(512, 14, 14)), 
        Flatten(),
        Dropout(p),
        Dense(256, activation='relu'),
        BatchNormalization(),
        Dropout(p),
        Dense(256, activation='relu'),
        BatchNormalization(),
        Dropout(p),
        Dense(10, activation='softmax')
        ]

In [5]:
p=0.8

In [6]:
bn_model = Sequential(get_bn_da_layers(p))
bn_model.compile(Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

Now we can train the model as usual, with pre-computed augmented data.

In [10]:
batch_size=32
conv_val_feat = load_array(path+'results/conv_val_feat.dat')

In [21]:
bn_model.fit(da_conv_feat, da_trn_labels, batch_size=batch_size, nb_epoch=1, 
             validation_data=(conv_val_feat, val_labels))

Train on 39786 samples, validate on 2531 samples
Epoch 1/1
39786/39786 [==============================] - 11s - loss: 2.2034 - acc: 0.3428 - val_loss: 1.0258 - val_acc: 0.7068


In [22]:
bn_model.optimizer.lr=0.01

In [23]:
bn_model.fit(da_conv_feat, da_trn_labels, batch_size=batch_size, nb_epoch=4, 
             validation_data=(conv_val_feat, val_labels))

Train on 39786 samples, validate on 2531 samples
Epoch 1/4
39786/39786 [==============================] - 11s - loss: 0.9770 - acc: 0.6591 - val_loss: 0.6947 - val_acc: 0.8048
Epoch 2/4
39786/39786 [==============================] - 11s - loss: 0.7256 - acc: 0.7600 - val_loss: 0.7459 - val_acc: 0.7645
Epoch 3/4
39786/39786 [==============================] - 11s - loss: 0.6100 - acc: 0.7999 - val_loss: 0.6642 - val_acc: 0.7819
Epoch 4/4
39786/39786 [==============================] - 11s - loss: 0.5454 - acc: 0.8243 - val_loss: 0.7047 - val_acc: 0.7776


In [24]:
bn_model.optimizer.lr=0.0001

In [25]:
bn_model.fit(da_conv_feat, da_trn_labels, batch_size=batch_size, nb_epoch=4, 
             validation_data=(conv_val_feat, val_labels))

Train on 39786 samples, validate on 2531 samples
Epoch 1/4
39786/39786 [==============================] - 11s - loss: 0.4911 - acc: 0.8443 - val_loss: 0.6935 - val_acc: 0.7748
Epoch 2/4
39786/39786 [==============================] - 11s - loss: 0.4638 - acc: 0.8513 - val_loss: 0.6788 - val_acc: 0.7799
Epoch 3/4
39786/39786 [==============================] - 11s - loss: 0.4323 - acc: 0.8620 - val_loss: 0.7301 - val_acc: 0.7807
Epoch 4/4
39786/39786 [==============================] - 11s - loss: 0.4121 - acc: 0.8690 - val_loss: 0.7050 - val_acc: 0.7661


Looks good - let's save those weights.

In [26]:
bn_model.save_weights(path+'models/da_conv8_1.h5')

### Pseudo labeling

We're going to try using a combination of [pseudo labeling](http://deeplearning.net/wp-content/uploads/2013/03/pseudo_label_final.pdf) and [knowledge distillation](https://arxiv.org/abs/1503.02531) to allow us to use unlabeled data (i.e. do semi-supervised learning). For our initial experiment we'll use the validation set as the unlabeled data, so that we can see that it is working without using the test set. At a later date we'll try using the test set.

To do this, we simply calculate the predictions of our model...

In [6]:
bn_model.load_weights(path+'models/da_conv8_1.h5')

In [11]:
val_pseudo = bn_model.predict(conv_val_feat, batch_size=batch_size)

...concatenate them with our training labels...

In [28]:
comb_pseudo = np.concatenate([da_trn_labels, val_pseudo])

In [29]:
comb_feat = np.concatenate([da_conv_feat, conv_val_feat])

...and fine-tune our model using that data.

In [23]:
bn_model.load_weights(path+'models/da_conv8_1.h5')

In [30]:
bn_model.fit(comb_feat, comb_pseudo, batch_size=batch_size, nb_epoch=1, 
             validation_data=(conv_val_feat, val_labels))

Train on 42317 samples, validate on 2531 samples
Epoch 1/1
42317/42317 [==============================] - 12s - loss: 0.4503 - acc: 0.8630 - val_loss: 0.6764 - val_acc: 0.8009


In [31]:
bn_model.fit(comb_feat, comb_pseudo, batch_size=batch_size, nb_epoch=4, 
             validation_data=(conv_val_feat, val_labels))

Train on 42317 samples, validate on 2531 samples
Epoch 1/4
42317/42317 [==============================] - 12s - loss: 0.4254 - acc: 0.8696 - val_loss: 0.7533 - val_acc: 0.8068
Epoch 2/4
42317/42317 [==============================] - 12s - loss: 0.4100 - acc: 0.8745 - val_loss: 0.7116 - val_acc: 0.8021
Epoch 3/4
42317/42317 [==============================] - 12s - loss: 0.3948 - acc: 0.8813 - val_loss: 0.6923 - val_acc: 0.8021
Epoch 4/4
42317/42317 [==============================] - 12s - loss: 0.3770 - acc: 0.8867 - val_loss: 0.6607 - val_acc: 0.8321


In [32]:
bn_model.optimizer.lr=0.00001

In [33]:
bn_model.fit(comb_feat, comb_pseudo, batch_size=batch_size, nb_epoch=4, 
             validation_data=(conv_val_feat, val_labels))

Train on 42317 samples, validate on 2531 samples
Epoch 1/4
42317/42317 [==============================] - 12s - loss: 0.3705 - acc: 0.8881 - val_loss: 0.7115 - val_acc: 0.8080
Epoch 2/4
42317/42317 [==============================] - 12s - loss: 0.3672 - acc: 0.8923 - val_loss: 0.6606 - val_acc: 0.8163
Epoch 3/4
42317/42317 [==============================] - 12s - loss: 0.3581 - acc: 0.8961 - val_loss: 0.7194 - val_acc: 0.7890
Epoch 4/4
42317/42317 [==============================] - 12s - loss: 0.3474 - acc: 0.8980 - val_loss: 0.7105 - val_acc: 0.8151


That's a distinct improvement - even although the validation set isn't very big. This looks encouraging for when we try this on the test set.

In [34]:
bn_model.save_weights(path+'models/bn-ps8.h5')

### Submit

We'll find a good clipping amount using the validation set, prior to submitting.

In [7]:
def do_clip(arr, mx): return np.clip(arr, (1-mx)/9, mx)

In [8]:
bn_model.load_weights(path+'models/bn-ps8.h5')

In [12]:
val_preds = val_pseudo # val_preds is not defined...

In [13]:
keras.metrics.categorical_crossentropy(val_labels, do_clip(val_preds, 0.93)).eval()

array(0.6766369168652889)

In [14]:
conv_test_feat = load_array(path+'results/conv_test_feat.dat')

In [15]:
preds = bn_model.predict(conv_test_feat, batch_size=batch_size*2)

In [16]:
subm = do_clip(preds,0.93)

In [17]:
subm_name = path+'results/subm.gz'

In [20]:
classes = sorted(batches.class_indices, key=batches.class_indices.get)

In [21]:
test_filenames[:5]

['unknown/img_55736.jpg',
 'unknown/img_14468.jpg',
 'unknown/img_101887.jpg',
 'unknown/img_78833.jpg',
 'unknown/img_7982.jpg']

In [22]:
submission = pd.DataFrame(subm, columns=classes)
#submission.insert(0, 'img', [a[4:] for a in test_filenames])
submission.insert(0, 'img', [a[8:] for a in test_filenames])
submission.head()

,img,c0,c1,c2,c3,c4,c5,c6,c7,c8,c9
0,img_55736.jpg,0.007778,0.007778,0.930000,0.007778,0.007778,0.007778,0.043684,0.007778,0.007778,0.007778
1,img_14468.jpg,0.007778,0.007778,0.930000,0.007778,0.007778,0.007778,0.007778,0.007778,0.007778,0.007778
2,img_101887.jpg,0.007778,0.007778,0.930000,0.007778,0.007778,0.007778,0.007778,0.007778,0.007778,0.007778
3,img_78833.jpg,0.007778,0.007778,0.007778,0.009193,0.930000,0.007778,0.007778,0.007778,0.007778,0.007778
4,img_7982.jpg,0.007778,0.007778,0.930000,0.007778,0.007778,0.007778,0.007778,0.007778,0.020865,0.007778


In [23]:
submission.to_csv(subm_name, index=False, compression='gzip')

In [24]:
FileLink(subm_name)

/home/ubuntu/nbs/data/state/results/subm.gz

This gets 0.534 on the leaderboard.

## The "things that didn't really work" section

You can safely ignore everything from here on, because they didn't really help.

### Finetune some conv layers too

In [28]:
for l in get_bn_layers(p): conv_model.add(l)

In [29]:
for l1,l2 in zip(bn_model.layers, conv_model.layers[last_conv_idx+1:]):
    l2.set_weights(l1.get_weights())

In [30]:
for l in conv_model.layers: l.trainable =False

In [31]:
for l in conv_model.layers[last_conv_idx+1:]: l.trainable =True

In [36]:
comb = np.concatenate([trn, val])

In [37]:
gen_t = image.ImageDataGenerator(rotation_range=8, height_shift_range=0.04, 
                shear_range=0.03, channel_shift_range=10, width_shift_range=0.08)

In [38]:
batches = gen_t.flow(comb, comb_pseudo, batch_size=batch_size)

Exception: X (images tensor) and y (labels) should have the same length. Found: X.shape = (22424, 3, 224, 224), y.shape = (98208, 10)

In [176]:
val_batches = get_batches(path+'valid', batch_size=batch_size*2, shuffle=False)

Found 3478 images belonging to 10 classes.


In [177]:
conv_model.compile(Adam(lr=0.00001), loss='categorical_crossentropy', metrics=['accuracy'])

In [178]:
conv_model.fit_generator(batches, batches.N, nb_epoch=1, validation_data=val_batches, 
                 nb_val_samples=val_batches.N)

Epoch 1/1
22400/22424 [============================>.] - ETA: 0s - loss: 0.4348 - acc: 0.9200

MemoryError: Error allocating 1644167168 bytes of device memory (CNMEM_STATUS_OUT_OF_MEMORY).
Apply node that caused the error: GpuAllocEmpty(Shape_i{0}.0, Shape_i{0}.0, Elemwise{Composite{(((i0 - i1) // i2) + i2)}}[(0, 1)].0, Elemwise{Composite{(((i0 - i1) // i2) + i2)}}[(0, 1)].0)
Toposort index: 157
Inputs types: [TensorType(int64, scalar), TensorType(int64, scalar), TensorType(int64, scalar), TensorType(int64, scalar)]
Inputs shapes: [(), (), (), ()]
Inputs strides: [(), (), (), ()]
Inputs values: [array(128), array(64), array(224), array(224)]
Outputs clients: [[GpuDnnConv{algo='small', inplace=True}(GpuContiguous.0, GpuContiguous.0, GpuAllocEmpty.0, GpuDnnConvDesc{border_mode='valid', subsample=(1, 1), conv_mode='conv', precision='float32'}.0, Constant{1.0}, Constant{0.0})]]

HINT: Re-running with most Theano optimization disabled could give you a back-trace of when this node was created. This can be done with by setting the Theano flag 'optimizer=fast_compile'. If that does not 

In [ ]:
conv_model.optimizer.lr = 0.0001

In [ ]:
conv_model.fit_generator(batches, batches.N, nb_epoch=3, validation_data=val_batches, 
                 nb_val_samples=val_batches.N)

In [ ]:
for l in conv_model.layers[16:]: l.trainable =True

In [ ]:
conv_model.optimizer.lr = 0.00001

In [ ]:
conv_model.fit_generator(batches, batches.N, nb_epoch=8, validation_data=val_batches, 
                 nb_val_samples=val_batches.N)

In [ ]:
conv_model.save_weights(path+'models/conv8_ps.h5')

In [77]:
conv_model.load_weights(path+'models/conv8_da.h5')

In [135]:
val_pseudo = conv_model.predict(val, batch_size=batch_size*2)

In [159]:
save_array(path+'models/pseudo8_da.dat', val_pseudo)

### Ensembling

In [14]:
drivers_ds = pd.read_csv(path+'driver_imgs_list.csv')
drivers_ds.head()

,subject,classname,img
0,p002,c0,img_44733.jpg
1,p002,c0,img_72999.jpg
2,p002,c0,img_25094.jpg
3,p002,c0,img_69092.jpg
4,p002,c0,img_92629.jpg


In [15]:
img2driver = drivers_ds.set_index('img')['subject'].to_dict()

In [16]:
driver2imgs = {k: g["img"].tolist() 
               for k,g in drivers_ds[['subject', 'img']].groupby("subject")}

In [56]:
def get_idx(driver_list):
    return [i for i,f in enumerate(filenames) if img2driver[f[3:]] in driver_list]

In [17]:
drivers = driver2imgs.keys()

In [94]:
rnd_drivers = np.random.permutation(drivers)

In [95]:
ds1 = rnd_drivers[:len(rnd_drivers)//2]
ds2 = rnd_drivers[len(rnd_drivers)//2:]

In [68]:
models=[fit_conv([d]) for d in drivers]
models=[m for m in models if m is not None]

In [77]:
all_preds = np.stack([m.predict(conv_test_feat, batch_size=128) for m in models])
avg_preds = all_preds.mean(axis=0)
avg_preds = avg_preds/np.expand_dims(avg_preds.sum(axis=1), 1)

In [102]:
keras.metrics.categorical_crossentropy(val_labels, np.clip(avg_val_preds,0.01,0.99)).eval()

array(0.9753041572894531)

In [103]:
keras.metrics.categorical_accuracy(val_labels, np.clip(avg_val_preds,0.01,0.99)).eval()

array(0.6949396133422852, dtype=float32)